In [1]:
from tqdm import tqdm
from pathlib import Path
import pandas as pd
import numpy as np
import copy

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split

device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
# Default paths
ROOT = Path("dataset") # Root dataset directory

PATH_NO_LABEL = ROOT / "spotify_songs.csv"
PATH_INT_LABEL = ROOT / "spotify_songs_with_genre_int.csv"
# PATH_ONE_HOT_LABEL = ROOT / "filename.csv"

In [3]:
int_label_df = pd.read_csv(PATH_INT_LABEL)
int_label_df.head()

,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,genre_int
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,2oCs0DGTsRO98Gh5ZSl2Cx,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,-2.634,1,0.0583,0.1020,0.000000,0.0653,0.518,122.036,194754,0
1,0r7CVbZTWZgbTCYdfa2P31,Memories - Dillon Francis Remix,Maroon 5,67,63rPSO264uRjW1X5E6cWv6,Memories (Dillon Francis Remix),2019-12-13,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,-4.969,1,0.0373,0.0724,0.004210,0.3570,0.693,99.972,162600,0
2,1z1Hg7Vb0AhHDiEmnDE79l,All the Time - Don Diablo Remix,Zara Larsson,70,1HoSmj2eLcsrR0vE9gThr4,All the Time (Don Diablo Remix),2019-07-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,-3.432,0,0.0742,0.0794,0.000023,0.1100,0.613,124.008,176616,0
3,75FpbthrwQmzHlBJLuGdC7,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,1nqYsOef1yKKuGOVchbsk6,Call You Mine - The Remixes,2019-07-19,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,-3.778,1,0.1020,0.0287,0.000009,0.2040,0.277,121.956,169093,0
4,1e8PAfcKUYoKkxPhrHqw4x,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,7m7vv9wlQ4i0LFuJiE2zsQ,Someone You Loved (Future Humans Remix),2019-03-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,-4.672,1,0.0359,0.0803,0.000000,0.0833,0.725,123.976,189052,0


In [4]:
from sklearn.preprocessing import StandardScaler

# feature_col 변수가 아직 정의되지 않았을 때도 안전하게 동작하도록 컬럼 리스트를 직접 사용
features = ["track_popularity", "danceability", "energy", "key", "loudness", "mode", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "duration_ms"]

# 1) 라벨 진단 및 0-based 재매핑 (CrossEntropy는 0-based 정수 라벨 필요)
print('genre_int dtype, min, max:', int_label_df['genre_int'].dtype, int_label_df['genre_int'].min(), int_label_df['genre_int'].max())
print('Label distribution:\n', int_label_df['genre_int'].value_counts())
if int_label_df['genre_int'].min() != 0 or int_label_df['genre_int'].nunique() != (int_label_df['genre_int'].max() + 1):
    int_label_df['genre_int'], uniques = pd.factorize(int_label_df['genre_int'])
    print('Remapped labels to 0..C-1. #original labels:', len(uniques))

# 2) Feature 스케일링: duration_ms 같은 큰 값 때문에 초기 logits/손실이 매우 큼
scaler = StandardScaler()
int_label_df[features] = scaler.fit_transform(int_label_df[features])
print('Feature means (post-scale):', np.round(int_label_df[features].mean().values,3))
print('Feature stds  (post-scale):', np.round(int_label_df[features].std().values,3))

genre_int dtype, min, max: int64 0 5
Label distribution:
 genre_int
5    6043
1    5746
0    5507
3    5431
4    5155
2    4951
Name: count, dtype: int64
Feature means (post-scale): [-0.  0.  0. -0.  0. -0. -0.  0. -0.  0.  0.  0.  0.]
Feature stds  (post-scale): [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [5]:
feature_col = ["track_popularity", "danceability", "energy", "key", "loudness", "mode", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "duration_ms"]
label_col = ["genre_int"]

In [6]:
from sklearn.metrics import accuracy_score, f1_score

def evaluate(model, dataloader, device="cpu"):
    """
    Evaluate a classification model on a given dataset.

    Args:
        model (torch.nn.Module): The classification model to evaluate.
        dataloader (DataLoader): DataLoader providing batches of {"X": features, "y": labels}.
        device (str, optional): Device to run evaluation on ("cpu" or "cuda"). Default is "cpu".

    Returns:
        dict: A dictionary containing:
            - "accuracy": Overall accuracy of predictions.
            - "f1_macro": Macro-averaged F1 score across all classes.
    """
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for batch in dataloader:
            X = batch["X"].to(device)
            y = batch["y"].to(device)
            logits = model(X)
            preds = torch.argmax(logits, dim=1)
            all_preds.extend(preds.cpu().tolist())
            all_labels.extend(y.cpu().tolist())

    acc = accuracy_score(all_labels, all_preds)
    f1_macro = f1_score(all_labels, all_preds, average="macro", zero_division=0)

    return {"accuracy": acc, "f1_macro": f1_macro}

In [7]:
import torch.nn as nn

class Classifier(nn.Module):
    def __init__(self, in_dim, hidden=256, num_classes=6):
        super().__init__()
        self.fc1 = nn.Linear(in_dim, hidden)
        self.fc2 = nn.Linear(hidden, num_classes)
        # 가중치 초기화로 초기 출력 폭을 줄임
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.zeros_(self.fc1.bias)
        nn.init.xavier_uniform_(self.fc2.weight)
        nn.init.zeros_(self.fc2.bias)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        return self.fc2(x)

In [8]:
class ProductDataset(Dataset):
    def __init__(self, df, feature_cols=feature_col, label_col=label_col):
        self.X = torch.tensor(df[feature_cols].to_numpy(dtype="float32"))
        # label_col은 리스트이므로 2D가 될 수 있음 -> 1D로 변환
        self.y = torch.tensor(df[label_col].to_numpy().ravel(), dtype=torch.long)
        
    def __len__(self): 
        return len(self.y)
    
    def __getitem__(self, idx): 
        return {"X": self.X[idx], "y": self.y[idx]}

In [9]:
# === Prepare test dataset and data loader ===
dataset = ProductDataset(
    df=int_label_df,
    feature_cols=feature_col,
    label_col=label_col,
)

# === Split training dataset into train/validation sets (50:25:25) ===
val_ratio, test_ratio = 0.25, 0.25
val_size  = int(len(dataset) * val_ratio)
test_size  = int(len(dataset) * test_ratio)
train_size = len(dataset) - val_size - test_size

g = torch.Generator().manual_seed(42)
train_split, val_split, test_split = random_split(dataset, [train_size, val_size, test_size], generator=g)

# === Create DataLoaders for training and validation ===
train_loader = DataLoader(train_split, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_split,   batch_size=64, shuffle=False)
test_loader  = DataLoader(test_split, batch_size=64, shuffle=False)

In [10]:
# === Initialize model and optimizer ===
in_dim = next(iter(train_loader))["X"].shape[1]  # 13
num_classes = int(int_label_df['genre_int'].nunique())  # 정확한 라벨 개수
model = Classifier(in_dim=in_dim, hidden=256, num_classes=num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [11]:
def print_eval_result(metrics: dict, stage="val", is_improved=False):
    """
    Print evaluation results (accuracy, F1-macro).
    
    Args:
        metrics: dict with keys 'accuracy' and 'f1_macro'
        stage: string label (e.g., "val", "test")
        is_improved: mark with '*' if results improved
    """
    star = " *" if is_improved else ""
    print(f"[{stage.upper():4}] Acc: {metrics['accuracy']:.4f} | "
          f"F1-macro: {metrics['f1_macro']:.4f}{star}")

In [ ]:
# b = next(iter(train_loader))
# print("X:", b["X"].shape, b["X"].mean().item(), b["X"].std().item())
# print("y range:", int(torch.min(b["y"])), int(torch.max(b["y"])))

# c = next(iter(val_loader))
# print("X:", c["X"].shape, c["X"].mean().item(), c["X"].std().item())
# print("y range:", int(torch.min(c["y"])), int(torch.max(c["y"])))

X: torch.Size([32, 13]) 0.004765113815665245 1.027963399887085
y range: 0 5
X: torch.Size([64, 13]) -0.03529072180390358 0.9689816832542419
y range: 0 5


In [13]:
# === Training loop with validation, test evaluation, and early stopping ===

EPOCHS = 100

best_score = 0
wait_time = 6
cnt = 0
best_model_state = copy.deepcopy(model.state_dict())

train_losses, val_acc_list, test_acc_list = [], [], []

for epoch in range(1, EPOCHS + 1):
    # --- Training phase ---
    model.train()
    total_loss = 0.0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch}"):
        X, y = batch["X"].to(device), batch["y"].to(device)
        # if y.dim() == 2 and y.size(1) == 1:
        #     y = y.squeeze(1)          # (B,1) → (B,)
        logits = model(X)
        loss = F.cross_entropy(logits, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / max(1, len(train_loader))
    train_losses.append(avg_loss)
    print(f"[Epoch {epoch}] Train Loss: {avg_loss:.4f}")

    test_result = evaluate(model, test_loader, device=device)
    test_acc = float(test_result.get("accuracy", 0.0))
    test_acc_list.append(test_acc)
    print_eval_result(test_result, stage="test")
    
    val_result = evaluate(model, val_loader, device=device)
    val_acc = float(val_result.get("accuracy", 0.0))
    val_acc_list.append(val_acc)
    print_eval_result(val_result, stage="validation")
    print()

    if val_acc > best_score:
        best_score = val_acc
        best_model_state = copy.deepcopy(model.state_dict())
        cnt = 0
    else:
        cnt += 1
        if cnt >= wait_time:
            print(f"No imporvement for {cnt} Epochs")
            break

Epoch 1: 100%|██████████| 514/514 [00:01<00:00, 279.20it/s]


[Epoch 1] Train Loss: 1.3883
[TEST] Acc: 0.4976 | F1-macro: 0.4907
[VALIDATION] Acc: 0.4967 | F1-macro: 0.4902



Epoch 2: 100%|██████████| 514/514 [00:02<00:00, 243.61it/s]


[Epoch 2] Train Loss: 1.2902
[TEST] Acc: 0.5015 | F1-macro: 0.4961
[VALIDATION] Acc: 0.5037 | F1-macro: 0.4969



Epoch 3: 100%|██████████| 514/514 [00:12<00:00, 40.75it/s]


...

[Epoch 18] Train Loss: 1.1805
[TEST] Acc: 0.5313 | F1-macro: 0.5156
[VALIDATION] Acc: 0.5401 | F1-macro: 0.5244



Epoch 19: 100%|██████████| 514/514 [00:02<00:00, 197.85it/s]


[Epoch 19] Train Loss: 1.1746
[TEST] Acc: 0.5297 | F1-macro: 0.5130
[VALIDATION] Acc: 0.5359 | F1-macro: 0.5192



Epoch 20: 100%|██████████| 514/514 [00:02<00:00, 204.26it/s]


[Epoch 20] Train Loss: 1.1698
[TEST] Acc: 0.5347 | F1-macro: 0.5261
[VALIDATION] Acc: 0.5384 | F1-macro: 0.5293

No imporvement for 6 Epochs


In [14]:
# === Load the best model and evaluate on the test set ===
model.load_state_dict(best_model_state) 

final_test_result = evaluate(model, test_loader, device=device)
print_eval_result(final_test_result, stage="final_test")

[FINAL_TEST] Acc: 0.5330 | F1-macro: 0.5251


In [ ]:
# 스포티파이 api 이용해서 뭔가 해보려고 했으나 policy update로 인해 무산

# import pprint
# from spotifyyy import *

# genre_dict = {0: 'pop', 1: 'rap', 2: 'rock', 3: 'r&b', 4: 'latin', 5: 'edm'}
# query = input()

# respond = get_track_by_search(query)
# pprint.pprint(respond)

In [ ]:
# 그래서 걍 랜덤으로 한 행 뽑아서 테스트 해봄

genre_dict = {0: 'pop', 1: 'rap', 2: 'rock', 3: 'r&b', 4: 'latin', 5: 'edm'}
row = int_label_df.sample(n=1, random_state=1).iloc[0]

song_name   = row["track_name"]
artist_name = row["track_artist"]
answer      = row.get("playlist_genre", None)

# === 스케일러에 feature 이름을 유지해 전달 ===
x_df = pd.DataFrame([row[features]], columns=features)
x_np = scaler.transform(x_df)[0]

# === 모델 추론 ===
model.eval(); model.to(device)
x = torch.tensor(x_np, dtype=torch.float32).unsqueeze(0).to(device)

with torch.no_grad():
    logits = model(x)
    probs  = F.softmax(logits, dim=1)[0]
    pred_id = int(torch.argmax(probs).item())
    pred_conf = float(probs[pred_id].item())

pred_genre = genre_dict[pred_id]

print(f"song: {song_name} | artist: {artist_name}")
if answer is not None:
    print(f"answer: {answer} | pred: {pred_genre} | conf: {pred_conf:.4f}")
else:
    print(f"pred: {pred_genre} | conf: {pred_conf:.4f}")

# 확률 Top-k 장르들 보기
k = 3
top_p, top_i = torch.topk(probs, k)
print("top-k:", [(genre_dict[int(i)], float(p)) for p, i in zip(top_p, top_i)])

song: Poetic Justice | artist: Kendrick Lamar
answer: rap | pred: rap | conf: 0.9963
top-k: [('rap', 0.9962978959083557), ('r&b', 0.0036934015806764364), ('latin', 8.665314453537576e-06)]
